In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Sample list of URLs to scrape
urls = ['https://www.newhaven.edu/student-life/get-involved/faq.php',
       'https://www.newhaven.edu/student-life/career-development-center/faq.php',
       'https://www.newhaven.edu/student-life/diversity-inclusion/faq.php',
       'https://www.newhaven.edu/about/departments/one-stop/faq.php',
       'https://www.newhaven.edu/families/faq.php',
       'https://www.newhaven.edu/student-life/student-affairs/faq.php',
       'https://www.newhaven.edu/academics/study-abroad/faq.php',
       'https://www.newhaven.edu/admissions/undergraduate/international/faq/index.php',
       'https://www.newhaven.edu/about/facts-figures.php',
       'https://www.newhaven.edu/about/departments/bursars/faq.php',
       'https://www.newhaven.edu/admissions/financial-aid/graduate/faq.php',
       'https://www.newhaven.edu/student-life/graduate-and-international/faq.php'
]

In [ ]:
def scrape_qa_from_website(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        questions = soup.find_all('span', class_="accordion-link-title")
        answers = soup.find_all('div', class_='panel')

        # Ensure questions and answers are paired correctly even if lengths differ
        min_length = min(len(questions), len(answers))
        extracted_questions = [question.text.strip() for question in questions[:min_length]]
        extracted_answers = [answer.text.strip() for answer in answers[:min_length]]

        # Handle missing data by adding placeholders for shorter lists
        diff_length = abs(len(questions) - len(answers))
        if len(questions) > len(answers):
            extracted_answers.extend(['Missing Answer'] * diff_length)
        elif len(answers) > len(questions):
            extracted_questions.extend(['Missing Question'] * diff_length)

        return extracted_questions, extracted_answers
    else:
        print(f"Failed to fetch content from {url}")
        return [], []


In [ ]:
# Scraping data from multiple websites
all_questions = []
all_answers = []

for url in urls:
    questions, answers = scrape_qa_from_website(url)
    all_questions.extend(questions)
    all_answers.extend(answers)

# Creating a DataFrame using pandas
data = {
    'Questions': all_questions,
    'Answers': all_answers
}

df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)
# df.to_csv('questions_answers.csv', index=False)  # Uncomment to save the data to a CSV file


                                             Questions  \
0    Is there a list of Recognized Student Organiza...   
1            How can my commuter student get involved?   
2    Where can my commuter student go in between cl...   
3    What leadership opportunities are available fo...   
4    How does my student apply for leadership posit...   
..                                                 ...   
145  Does the University provide bus passes to use ...   
146  Are students required to live on campus? If no...   
147  I recently moved. Do I need to report my chang...   
148  How can I connect with other students once I a...   
149         Is the University of New Haven accredited?   

                                               Answers  
0    The Center for Student Engagement, Leadership ...  
1    All programs, events, and Recognized Student O...  
2    There are multiple locations on campus that of...  
3    Take Charge is the University of New Haven’s l...  
4    Applications 

In [ ]:
# df.to_csv('questions_answers.csv', index=False)  # Uncomment to save the data to a CSV file